In [1]:
JNA=1
FEB=2
MAR=3
NOV=11
DEC=12

In [3]:
from enum import Enum
Month=Enum('Month',('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'))

In [15]:
for name, member in Month.__members__.items():
    print(name,'=>',member,',',member.value)
print(Month.__members__.items())

Jan => Month.Jan , 1
Feb => Month.Feb , 2
Mar => Month.Mar , 3
Apr => Month.Apr , 4
May => Month.May , 5
Jun => Month.Jun , 6
Jul => Month.Jul , 7
Aug => Month.Aug , 8
Sep => Month.Sep , 9
Oct => Month.Oct , 10
Nov => Month.Nov , 11
Dec => Month.Dec , 12
odict_items([('Jan', <Month.Jan: 1>), ('Feb', <Month.Feb: 2>), ('Mar', <Month.Mar: 3>), ('Apr', <Month.Apr: 4>), ('May', <Month.May: 5>), ('Jun', <Month.Jun: 6>), ('Jul', <Month.Jul: 7>), ('Aug', <Month.Aug: 8>), ('Sep', <Month.Sep: 9>), ('Oct', <Month.Oct: 10>), ('Nov', <Month.Nov: 11>), ('Dec', <Month.Dec: 12>)])


In [5]:
from enum import Enum,unique
@unique
class Weekday(Enum):
    Sun=0
    Mon=1
    Tue=2
    Wed=3
    Thu=4
    Fri=5
    Sat=6

In [9]:
day1=Weekday.Mon
print(day1)

Weekday.Mon


In [10]:
print(Weekday.Tue)

Weekday.Tue


In [11]:
print(Weekday['Tue'])

Weekday.Tue


In [12]:
print(Weekday.Tue.value)

2


In [16]:
print(day1==Weekday.Mon)

True


In [17]:
print(day1==Weekday.Tue)

False


In [18]:
print(day1==Weekday(1))    

True


In [19]:
print(Weekday(1))                   #根据value的值获取枚举常量

Weekday.Mon


In [20]:
Weekday(7)

ValueError: 7 is not a valid Weekday

In [21]:
for name,member in Weekday.__members__.items():
    print(name,'=>',member)

Sun => Weekday.Sun
Mon => Weekday.Mon
Tue => Weekday.Tue
Wed => Weekday.Wed
Thu => Weekday.Thu
Fri => Weekday.Fri
Sat => Weekday.Sat


In [28]:
# -*- coding: utf-8 -*-
from enum import Enum, unique
@unique
class Gender(Enum):
    Male = 0
    Female = 1

class Student(object):
    def __init__(self, name, gender):
        self.name = name
        if isinstance(gender,Gender):
            self.gender=gender
        else:
            raise ValueError('只允许接受Gender类型的参数')
        
bart = Student('Bart', Gender.Male)
if bart.gender == Gender.Male:
    print('测试通过!')
else:
    print('测试失败!')

测试通过!


In [29]:
#动态语言和静态语言最大的不同，就是函数和类的定义，不是编译时定义的，而是运行时动态创建的。

In [35]:
#type()用于创建class
#type('class的名称','继承的父类集合','class的方法名称与函数绑定')

In [1]:
def fn(self, name='world'): # 先定义函数
    print('Hello, %s.' % name)
    
Hello = type('Hello', (object,), dict(hello=fn)) # 创建Hello class

In [2]:
#元类
class ListMetaclass(type):
    def __new__(cls,name,bases,attrs):
        attrs['add']=lambda self,value:self.append(value)
        return type.__new__(cls,name,bases,attrs)

In [6]:
class MyList(list,metaclass=ListMetaclass):
    pass
L=MyList()
L.add(1)
L

[1]

In [8]:
class Field(object):

    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type

    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)

In [9]:
class StringField(Field):
    def __init__(self,name):
        super(StringField,self).__init__(name,'varchar(100)')
class IntegerField(Field):
    def __init__(self,name):
        super(IntegerField,self).__init__(name,'bigint')

In [10]:
class ModelMetaclass(type):
    def __new__(cls,name,bases,attrs):
        if name=='Model':
            return type.__new__(cls,name,bases,attrs)
        print('Found model:%s'%name)
        mappings=dict()
        for k,v in attrs.items():
            if isinstance(v,Field):
                print('Found mapping:%s==>%s'%(k,v))
                mappings[k]=v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__']=mappings
        attrs['__table__']=name
        return type.__new__(cls,name,bases,attrs)

In [16]:
class Model(dict,metaclass=ModelMetaclass):
    def __init__(self,**kw):
        super(Model,self).__init__(**kw)
    def __getatter__(self,key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'"%key)
    def __setattr__(self,key,value):
        self[key]=value
    def save(self):
        fields=[]
        params=[]
        args=[]
        for k,v in self.__mappings__items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self,k,None))
        sql='insert into %s(%s) values (%s)'%(self.__table,','.join(fields),','.join(params))
        print('SQL:%s'% sql)
        print('ARGS:%s'%str(agrs))

In [17]:
class User(Model):
    id=IntegerField('id')
    name=StringField('username')
    email=StringField('email')
    password=StringField('password')
    
u=User(id=12345,name='Michael',email='test@irm.org',password='mt-pwd')
u.save()

Found model:User
Found mapping:id==><IntegerField:id>
Found mapping:name==><StringField:username>
Found mapping:email==><StringField:email>
Found mapping:password==><StringField:password>


AttributeError: 'User' object has no attribute '_Model__mappings__items'